# Transfering Spotify playlist to YouTube

With the following code, you will be able to transfer a playlist from Spotify to Youtube. 



Make sure that you already have the following, you will need it.

- client_secret.json (you get it in the OAuth 2.0 Client IDs (inside the Google Cloud Console) )
- api_key (inside the Google Cloud Console)
- client_id & client_secret (inside Spotify Developer Platform)



Basically, the code does the following:

- Authenticate to YouTube
  - For this, you will need to have a .json file with some credentials. You get this json file on the console of your application when you enable OAuth 2.0 authentication. (You need to get into the user in order to manage its playlists.)
- Authenticate to Spotify
  - You have to pass the client_id & client_secret that you get on Spotify's developer platform when you create your app. (Playlists should be public because we are not requiring user authentication).
- Get all tracks of a given user & playlist_id and parse them into strings. 
- Loop the strings you just made into the YouTube API to search and get the video id of the first result (hopefully, the first result will always be the best/official version of the song). 
- Insert that video id into a playlist in YouTube.

In [1]:
import os
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from googleapiclient.discovery import build

import spotipy
from spotipy.oauth2 import SpotifyOAuth


#youtube api key (not real creds)
api_key = 'FASDq2134123asdfasdfasdfA123412341234ASD'
client_path = "your_path/client_secret.json"

#spotify creds (not real creds)
client_id = 'dfa8sd70fas8d7fas8df70as8d7fas8'
client_secret = 'dfa8sd70fas8d7fas8df70as8d7fas8'

In [2]:
def youtube_auth(client_path):
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
    
    scopes = ["https://www.googleapis.com/auth/youtube.force-ssl"]

    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = client_path

    # Get credentials and create an API client
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, scopes)
    credentials = flow.run_console()
    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, credentials=credentials)
    
    return youtube


def get_playlist_tracks(username, s_playlist_id, playlist_name):
    """
    Function get tracks from Spotify
    """
    print("Getting tracks for:", playlist_name)
    results = sp.user_playlist_tracks(username,s_playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks


def get_playlist_namedesc(s_playlist_id):
    """
    Function gets playlist name and description from Spotify
    """
    results = sp.playlist(s_playlist_id)
    return results['name'], results['description']


def insert_video_to_new_playlist(video_ids, names, playlist_id):
    """
    Function that inserts the video_ids into the playlist_id.
    video_ids = list of strings
    playlist_id = string
    """
    for i, vid in enumerate(video_ids):
        request = youtube.playlistItems().insert(
            part="snippet",
            body={
              "snippet": {
                "playlistId": playlist_id,
                "position": 0,
                "resourceId": {
                  "kind": "youtube#video",
                  "videoId": vid
                }
              }
            }
        )
        response = request.execute()
        print('Added:', names[i])
        time.sleep(1)
    print('All done.', len(video_ids), 'songs added to YouTube')
    
    
def create_playlist(playlist_name, playlist_description):
    """
    Function creates a new playlists and returns its id
    """
    request = youtube.playlists().insert(
    part="snippet,status",
    body={
      "snippet": {
        "title": playlist_name,
        "description": playlist_description,
        "defaultLanguage": "en"
      },
      "status": {
        "privacyStatus": "public"
      }
    }
    )
    new_playlist = request.execute()
    
    return new_playlist['id'] 
        
def get_spotity_titles_in_youtube(final_list):
    """
    Function to use YouTube's API to search titles to get video id
    """
    ids = []
    names = []
    for item in final_list:
        request = youtube.search().list(
                part="snippet",
                maxResults=1,
                q=item
            )

        response = request.execute()
        ids.append(response['items'][0]['id']['videoId'])
        names.append(response['items'][0]['snippet']['title'])
    return ids, names


def making_strings(tracks, playlist_name):
    """
    Function creates a string like 'name of artist - name of song'
    """
    final_list = []
    for i, track in enumerate(tracks):
        string = ''
        string += tracks[i]['track']['artists'][0]['name']
        string += ' - '
        string += tracks[i]['track']['name']
        final_list.append(string)
        
    print('DONE! Got',len(final_list), 'tracks')
    print('')
    return final_list


def find_youtube_link(link):
    """
    Function extracts video id from url
    """
    try:
        return re.search(r'v=([\w-]+)', link).group(1)
    except:
        return np.nan


In [3]:
youtube = youtube_auth(client_path)


sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri="https://www.spotify.com/mx/home/",
                                               scope="user-library-read"))

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=384828426045-2ddfkkcaul8ebr4ufc5esjlihau0m572.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=hSphQMDiBg1J91Lsp3Ycq6D9VgtF9z&prompt=consent&access_type=offline
Enter the authorization code: 4/1AY0e-g4-5YVNSvxh_OsDxFnA509Lmro7bYtuXQoAPmuCy4olWu7V4QCmYzA


In [4]:
#playlist to get
username = 'spotify:user:1111111111111'
s_playlist_id = 'spotify:playlist:2rEWSyasdfass6Bdwml6Ks'


# Getting playlist name and description (from Spotify)
playlist_name, playlist_description = get_playlist_namedesc(s_playlist_id)

# Getting tracks in playlist (from Spotify)
tracks = get_playlist_tracks(username,s_playlist_id, playlist_name)


# Making strings to search in YouTube
final_list = making_strings(tracks, playlist_name)


# Use YouTube's API to get video id from tracks
ids, names = get_spotity_titles_in_youtube(final_list)


# Create a new playlist on YouTube with the same name as it had on Spotify
new_playlist_id = create_playlist(playlist_name, playlist_description)


# Insert tracks into YouTube playlist
insert_video_to_new_playlist(ids, names, new_playlist_id)


Getting tracks for: Ricas Sabrosas
DONE! Got 15 tracks

Added: Audioslave - Show Me How to Live (Official Video)
Added: Red Hot Chili Peppers - Tell Me Baby [Official Music Video]
Added: The Offspring - Pretty Fly (For A White Guy) (Official Music Video)
Added: Green Day - When I Come Around [Official Music Video]
Added: Royal Blood - Little Monster (Official Video)
Added: Crobot - Nowhere to Hide (Official)
Added: Beware of Darkness - Howl
Added: Muse - Psycho [Official Lyric Video]
Added: Red Hot Chili Peppers - Dani California [Official Music Video]
Added: Rise Against - Re-Education (Through Labor) (Uncensored) [Official Video]
Added: Red Hot Chili Peppers - Give It Away [Official Music Video]
Added: Rage Against The Machine - Guerrilla Radio (Official Music Video)
Added: The Kooks - Naive
Added: Stone Temple Pilots - Interstate Love Song (Official HD Music Video)
Added: Red Hot Chili Peppers - Scar Tissue [Official Music Video]
All done. 15 songs added to YouTube
